In [ ]:
#!pip install --upgrade pip setuptools wheel
#!python -m pip install opencv-python
#!pip install matplotlib
#!pip freeze
#!pip install opencv-python-headless
#!pip install scikit-image
#!pip install -U scikit-fuzzy
#!pip install pandas
#!pip install scikit-learn
#!pip install xgboost
#!pip install seaborn

import os
import cv2
import library
import numpy as np
import hair_removal
import pandas as pd
import glcm_features
import color_features
import shape_features
import texture_features
from os.path import join
from multiprocessing import Process
from matplotlib import pyplot as plt

### GENERATING FEATURES

In [ ]:
current_challenge = "ch1"
pipeline = "F"

def run_process(args):
    flag = True
    dictF = {}
    features = pd.DataFrame()
    
    idx = args[0]
    samples = args[1]
    current_challenge = args[2]
    
    for sample in samples:
        
        # pipeline call
        #features = library.PipelineB(dictF, features, sample, current_challenge)
        #features = library.PipelineC(dictF, features, sample, current_challenge)
        #features = library.PipelineD(dictF, features, sample, current_challenge)
        #features = library.PipelineE(dictF, features, sample, current_challenge)
        #features = library.PipelineF(dictF, features, sample, current_challenge)
        #features = library.PipelineG(dictF, features, sample, current_challenge)
        
        
        
        # save features
        library.writeFeatures(features,
                      flag,
                      os.path.join('/home',
                         'emily',
                         'Desktop',
                         'CAD',
                         'MelanomaChallenge',
                         'features'),
                         'features{}_{}_{}.csv'.format(current_challenge, pipeline, idx))
        
        flag = False
        features = pd.DataFrame()
        dictF.clear()


In [ ]:
all_processes = []
BATCH_SIZE = 4

def create_batches(iterable, n = 1):
    current_batch = []
    for item in iterable:
        current_batch.append(item)
        if len(current_batch) == n:
            yield current_batch
            current_batch = []
    if current_batch:
        yield current_batch

def prepare_process(samples):
    # divide the samples into n=8 subsets
    # the n value depend on the capability of the pc and the total quantity of samples
    subsetsamples = np.array_split(samples, 8)#14800/8
    for idx, subsetsample in enumerate(subsetsamples):
        p = Process(target=run_process, args=([idx, subsetsample, current_challenge], ))
        all_processes.append(p)
    
    for batch in create_batches(all_processes,BATCH_SIZE):
        for process in batch:
            process.start()
        for process in batch:
            process.join()

flag = True
output = ""

# get a sample of the path
allsamples, flag = library.get_sample("/home/emily/Desktop/CAD/challenge1/train",output, flag, 14800)

# get names belonging to already performed samples
if os.path.exists(os.path.join('/home',
                     'emily',
                     'Desktop',
                     'CAD',
                     'MelanomaChallenge',
                     'features',
                     'features{}_{}.csv'.format(current_challenge, pipeline))):
    donesamples = pd.read_csv(os.path.join('/home',
                         'emily',
                         'Desktop',
                         'CAD',
                         'MelanomaChallenge',
                         'features',
                         'features{}_{}.csv'.format(current_challenge, pipeline)))['name'].values.tolist()

    # get names of missing samples
    missingsamples = library.uniques_names(allsamples, donesamples)

    # if want to perform the samples pending
    prepare_process(missingsamples)
else:
    # if want to perform all the samples
    prepare_process(allsamples)